In [42]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

In [43]:
train_features = pd.read_csv("train_features.csv")
test_features = pd.read_csv("test_features.csv")
train_labels = pd.read_csv("train_labels.csv")


### Train set data
X_train_features = train_features.drop(columns=["pid"])
X_train_pid = train_features.get("pid") ## just in case if needed
y_train_labels = train_labels.drop(columns = ["pid"])
train_label_pid = train_labels.get("pid") ## just in case if needed

### Test set data
X_test_features = test_features.drop(columns = ["pid"])
X_test_pid = test_features.get("pid")


In [44]:
def extract_pids(feat_dataframe):
    
    
    all_pids = feat_dataframe["pid"].unique()
    return all_pids
    

In [45]:
all_pid = extract_pids(train_features)

train_pid, val_pid = train_test_split(all_pid, train_size= 0.8)

In [46]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

In [47]:
imputer = IterativeImputer(missing_values=np.NaN,n_nearest_features=4)
mask = train_features['pid'].isin(train_pid)
oo = train_features.loc[mask]#.drop(columns = ["pid"])
idf=pd.DataFrame(imputer.fit_transform(oo))
idf.columns=oo.columns
idf.index=oo.index
mask_kk = train_labels['pid'].isin(train_pid)
kk = train_labels.loc[mask_kk]


mask2 = train_labels['pid'].isin(val_pid)
val_labels = train_labels.loc[mask2].drop(columns = ["pid"]).to_numpy()
val_features = train_features['pid'].isin(val_pid).drop(columns = ["pid"])

mask3 = train_features['pid'].isin(val_pid)
pp = train_features.loc[mask3]#.drop(columns = ["pid"])
idf_val=pd.DataFrame(imputer.fit_transform(pp))
idf_val.columns=pp.columns
idf_val.index=pp.index


C:\Users\Ketzel\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\Ketzel\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [48]:
print(list(kk), kk['pid'].to_numpy()[0:10], idf['pid'].to_numpy()[40:50])
#print(kk[['pid','LABEL_Sepsis']])
print(train_pid.shape)

['pid', 'LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2', 'LABEL_Sepsis', 'LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate'] [    1    10   100  1000 10002 10006 10007 10009  1001 10010] [ 1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000. 10002. 10002.]
(15196,)


In [49]:
def impute_missing_data(dataframe):
    for (columnName, columnData) in dataframe.iteritems():
        numb_rows = columnData.shape
        not_NaN = columnData.count()
        if not_NaN==0: continue
        elif not_NaN == 1:
            dataframe[columnName] = columnData.fillna(columnData.mean())
        else:
            dataframe[columnName] = columnData.interpolate()
    return dataframe

In [50]:
def create_features(patient_data):
    relevant_data = patient_data
    #relevant_data = patient_data.drop(columns=["Time","Age","pid"])
    feat1 = relevant_data.mean().to_numpy().reshape(1,-1)
    feat2 = relevant_data.std().to_numpy().reshape(1,-1)
    feat3 = relevant_data.max().to_numpy().reshape(1,-1)
    patient_features = np.hstack((feat1,feat2))
    return patient_features
    

In [51]:
features = create_features(idf[idf['pid'] == train_pid[0]].drop(columns=["Time","Age","pid"]))
labels = train_labels[train_labels['pid'] == train_pid[0]].drop(columns=["pid"]).to_numpy().reshape(1,-1)

In [52]:
print(labels)

[[  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   19.
   91.8  95.7 108.2]]


In [53]:
for pid in train_pid[1:]:
    X_pid = idf[idf['pid'] == pid].drop(columns=["Time","Age","pid"])
    #X_pid = features[features[:,0]==pid,:]
    #X_pid = impute_missing_data(X_pid)
    X_feat = create_features(X_pid)
    features = np.vstack((features,X_feat))
    y_pid = train_labels[train_labels['pid'] == pid].drop(columns=["pid"])
    labels= np.vstack((labels,y_pid))
    
print(features.shape, y_pid.shape, labels.shape)

    
    
    


(15196, 68) (1, 15) (15196, 15)


In [54]:
val_features = create_features(idf_val[idf_val['pid'] == val_pid[0]].drop(columns=["Time","Age","pid"]))
val_labels = train_labels[train_labels['pid'] == val_pid[0]].drop(columns=["pid"]).to_numpy().reshape(1,-1)

In [55]:
for pid in val_pid[1:]:
    X_pid = idf_val[idf_val['pid'] == pid].drop(columns=["Time","Age","pid"])
    #X_pid = features[features[:,0]==pid,:]
    #X_pid = impute_missing_data(X_pid)
    X_feat = create_features(X_pid)
    val_features = np.vstack((val_features,X_feat))
    y_pid = train_labels[train_labels['pid'] == pid].drop(columns=["pid"])
    val_labels= np.vstack((val_labels,y_pid))
    
print(val_features.shape, y_pid.shape, val_labels.shape)

(3799, 68) (1, 15) (3799, 15)


In [56]:
print(val_labels[0,1:10])

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [57]:
from sklearn import svm

In [58]:

clf = svm.SVC(kernel='sigmoid', probability= True)
clf.fit(features, labels[:,1])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='sigmoid',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [59]:
y_hat = clf.predict_proba(val_features[:,:])

In [60]:
test_pid = extract_pids(test_features)
mask_test = test_features['pid'].isin(test_pid)
ii = test_features.loc[mask_test]#.drop(columns = ["pid"])
idf_test=pd.DataFrame(imputer.fit_transform(ii))
idf_test.columns=ii.columns
idf_test.index=ii.index

C:\Users\Ketzel\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:638: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [61]:
t_features = create_features(idf_test[idf_test['pid'] == test_pid[0]].drop(columns=["Time","Age","pid"]))

In [62]:
for pid in test_pid[1:]:
    X_pid = idf_test[idf_test['pid'] == pid].drop(columns=["Time","Age","pid"])
    #X_pid = features[features[:,0]==pid,:]
    #X_pid = impute_missing_data(X_pid)
    X_feat = create_features(X_pid)
    t_features = np.vstack((t_features,X_feat))

    
print(t_features.shape)

    
    
    


(12664, 68)


In [63]:
from sklearn.neural_network import MLPClassifier

In [64]:
tz =kk[['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']]
print(tz.to_numpy().shape)
## ['pid', 'LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2', 'LABEL_Sepsis', 'LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

(15196, 10)


In [65]:
test_labels = test_pid
model = MLPClassifier(activation='logistic')
model.fit(features, labels[:,0:10])
#model.fit(features, kk[['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']].to_numpy()) ####
y_hat = model.predict_proba(t_features[:,:])
test_labels = np.column_stack((test_labels,y_hat))

C:\Users\Ketzel\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [66]:
print(test_pid.shape)

(12664,)


In [67]:

ss = kk['LABEL_Sepsis'].to_numpy()
print(ss.shape, features.shape)
print(ss[ss==1])

(15196,) (15196, 68)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [68]:
clf = svm.SVC(kernel='linear', probability= False)
#clf.fit(features, labels[:,10] ) kk['pid'].to_numpy()
clf.fit(features, ss )
y_hat = clf.predict(t_features[:,:])
test_labels = np.column_stack((test_labels,y_hat))

In [69]:
from sklearn.linear_model import LinearRegression

In [70]:
reg = LinearRegression(fit_intercept= False, normalize= False)
y_hat = reg.fit(features, labels[:,11:]).predict(t_features[:,:])
test_labels = np.column_stack((test_labels,y_hat))

In [71]:
print(test_labels.shape, train_labels.to_numpy().shape)

(12664, 16) (18995, 16)


In [72]:
test_labels = pd.DataFrame(test_labels)
test_labels.columns=train_labels.columns
results = test_labels

In [76]:
###result = pd.concat([X_test_pid,pd.DataFrame(__result here__)],axis=1)
result = pd.concat([X_test_pid.drop_duplicates()],axis=1)

pd.DataFrame(results).to_csv("submit_Ketzel.csv", index=False, header=True)
results.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')


In [74]:
Vita_values = train_features.get(["pid","Heartrate", "SpO2", "ABPs", "ABPm", "ABPd", "RRate", "Temp" ])
display(Vita_values)

,pid,Heartrate,SpO2,ABPs,ABPm,ABPd,RRate,Temp
0,1,94.0,100.0,142.0,84.0,61.0,16.0,36.0
1,1,99.0,100.0,125.0,81.0,62.5,16.0,36.0
2,1,92.0,100.0,110.0,74.0,59.0,18.0,36.0
3,1,88.0,100.0,104.0,66.0,49.5,18.0,37.0
4,1,81.0,100.0,100.0,63.0,48.0,18.0,NaN
...,...,...,...,...,...,...,...,...
227935,9999,80.0,NaN,110.0,78.0,NaN,17.0,NaN
227936,9999,83.0,NaN,123.0,88.0,NaN,15.0,NaN
227937,9999,80.0,98.0,138.0,97.0,NaN,22.0,36.0
227938,9999,75.0,98.0,125.0,84.0,NaN,22.0,NaN


In [75]:
ss = Vita_values[0:30]
display(ss.std())
tt = Vita_values.groupby('pid').mean()
display(tt,tt.std())

pid          38.663535
Heartrate    14.810045
SpO2          1.448628
ABPs         16.009257
ABPm         17.880418
ABPd         17.205957
RRate         2.743555
Temp          0.767948
dtype: float64

,Heartrate,SpO2,ABPs,ABPm,ABPd,RRate,Temp
pid,,,,,,,
1,77.083333,100.000000,114.500000,68.333333,50.250000,17.000000,36.750000
2,59.000000,96.000000,131.181818,94.636364,70.090909,18.000000,36.000000
4,72.545455,99.272727,125.454545,80.909091,49.600000,14.636364,36.666667
6,87.333333,99.333333,100.666667,65.750000,48.833333,15.833333,37.166667
8,81.181818,97.800000,204.545455,143.900000,108.954545,17.181818,36.000000
...,...,...,...,...,...,...,...
31653,108.272727,96.181818,87.333333,65.636364,49.166667,15.545455,36.000000
31654,92.909091,96.090909,165.000000,122.363636,94.181818,16.909091,37.500000
31656,99.100000,100.000000,100.300000,78.600000,63.900000,24.900000,38.000000


Heartrate    16.135643
SpO2          2.227915
ABPs         19.454857
ABPm         13.587746
ABPd         11.667510
RRate         4.007461
Temp          0.670461
dtype: float64